In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 20% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_20, _ , y_20 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.2, stratify=y_label_0_1, random_state=42)


In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt20 = DecisionTreeClassifier()
dt20.fit(X_20, y_20)

RF


RandomForestClassifier()

In [7]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [24957    97  2277   853    27  2474   112 83075    73    26]
(24957, 56) (24957,)
Save 0 to baseline_Def1.npy
(97, 56) (97,)
Save 1 to baseline_Def2.npy
(2277, 56) (2277,)
Save 2 to baseline_Def3.npy
(853, 56) (853,)
Save 3 to baseline_Def4.npy
(27, 56) (27,)
Save 4 to baseline_Def5.npy
(2474, 56) (2474,)
Save 5 to baseline_Def6.npy
(112, 56) (112,)
Save 7 to baseline_Def8.npy
(83075, 56) (83075,)
Save 8 to baseline_Def9.npy
(73, 56) (73,)
Save 9 to baseline_Def10.npy
(26, 56) (26,)
Save 10 to baseline_Def11.npy
Execution Time: 1.686390 seconds
(341913,)
[0 2 3 4 5 8] [145258    182    518      1 173650  22304]
(145258, 56) (145258,)
Save 0 to BIM_Def1.npy
(182, 56) (182,)
Save 2 to BIM_Def3.npy
(518, 56) (518,)
Save 3 to BIM_Def4.npy
(1, 56) (1,)
Save 4 to BIM_Def5.npy
(173650, 56) (173650,)
Save 5 to BIM_Def6.npy
(22304, 56) (22304,)
Save 8 to BIM_Def9.npy
Execution Time: 2.696136 seconds
(341913,)
[0 2 3 4 5 7 8] [121738    167    393      

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}/UNSW_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
1,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
2,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
3,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
4,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
5,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
6,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
7,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
8,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
9,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
